## 导入Overlay

In [8]:
from pynq import Overlay
overlay = Overlay('./cordic.bit')

## cordic IP上层驱动

In [9]:
from pynq import DefaultIP

class cordicDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['xilinx.com:hls:cordic:1.0']

    def cal_sin(self, theta):
        self.write(0x10, theta)
        self.write(0x00,0b1)
        return self.read(0x18)
    
    def cal_cos(self, theta):
        self.write(0x10, theta)
        self.write(0x00, 0b1)
        return self.read(0x28)

# Register level driver
 - 直接读写寄存器来使用overlay中的hls ip
 - theta的数据类型为ap_fixed<28,8>

In [10]:
# theta = 0b0011_1100_0000_0000_0000_0000_0000  #60的定点数表示
theta = 0b0010_1101_0000_0000_0000_0000_0000  #45的定点数表示
# theta = 0b0101_1010_0000_0000_0000_0000_0000  #90的定点数表示

cordic = overlay.cordic_0
cordic.write(0x10, theta)  #向theta寄存器写值
cordic.write(0x00, 0b1)    #启动IP

In [11]:
#输出正弦和余弦值，输出值为ap_fixed<28.8>定点数直接换算为十进制的数值
cos = cordic.read(0x28)
sin = cordic.read(0x18)
print(cos)
print(sin)

741467
741444


## 使用上层驱动计算

In [12]:
cordic.cal_sin(0b0001_1110_0000_0000_0000_0000_0000)    #sin(30°)

524330

In [13]:
cordic.cal_cos(0b0011_1100_0000_0000_0000_0000_0000)   #cos(60°)

524330

## 性能分析
计算IP运行时间

In [14]:
import time

cordic.write(0x00, 0b1)
start_time = time.time()
while True:
    reg = cordic.read(0x00)
    if reg != 1:
        break
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0008602142333984375s
